<a href="https://colab.research.google.com/github/amadeu100401/Clusterizacao_Machine_Learning/blob/main/Atividade_clusteriza%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados Criminais

O objetivo dessa tarefa é construir um mapa do crime no Brasil a partir de dados abertos. O trabalho pode ser feito em dupla ou individual. Cada dupla ou pessoa deve fazer uma análise e pré-processamento dos dados para que possam construir um mapa do crime do Brasil. Além disso, vocês devem aplicar algoritmos de de clusterização para agrupar cidades de acordo com o nível de criminalidade (como o exemplo dos Estados Unidos que apresentei em sala de aula).

A base que vocês devem utilizar está disponível neste link:

http://dados.mj.gov.br/dataset/sistema-nacional-de-estatisticas-de-seguranca-publica

A base contém dados de crimes cometidos nos estados brasileiros. Na página vocês vão encontrar os dados por estado e por município. No entanto, para esse trabalho vamos usar apenas a base de estados. O nome do arquivo é `indicadoressegurancapublicauf.xlxs` e pode ser acessados no link acima onde tem **Dados Nacionais de Segurança Pública - UF**. No link também tem o arquivo **Dicionário de Dados - UF** que explica a tabela que vocês vão trabalhar. Além disso, o link contém outras informações relacionadas ao tema. Importante a leitura para ajudar na análise de vocês. 

O trabalho deve conter as seguintes tarefas

* pré-processamento e análise dos dados;
* aplicação dos algoritmos de clusterização e análise dos resultados;
* visualização dos resultados da análise e da clusterização. 

Para as etapas 1 e 3 vocês podem usar qualquer ferramenta: excel, power bi .... Para a etapa 2, utilizem os algoritmos de clusterização apresentados em sala de aula através da biblioteca **scikit-learn**. No material de aula mostro como construir uma tabela com os dados da clusterização. Essa saída pode ser usada nas ferramentas que vocês escolherem. Na etapa de clusterização vocês devem gerar o cluster e fazer a análise dos resultados. Variem parâmetros relacionados aos algoritmos de cluster para encontrar o melhor agrupamento. Essa análise deve ser feita também utilizando as métricas de avaliação não-supervisionadas vistas em sala de aula.

A seguir mostro como fazer a leitura do arquivo.

Bom trabalho ;) 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Imports necessários
import pandas as pd
from sklearn import preprocessing
from scipy.spatial import distance_matrix
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import plotly.graph_objects as go

## Carregando a base do Excel 

O arquivo de entrar é um `.xlsx` que possui duas planilhas. A primeira com as ocorrências criminais e a segunda com o número de vítimas de diferentes registros criminais. Cada planilha foi salva em um DataFrame distinto. Vocês devem analisar a melhor forma de utilizar esses dados durante a etapa de pré-processamento ;) 

In [4]:
data_ocorrencias = pd.read_excel("/content/drive/MyDrive/Análise de dados/UND 2/Trabalho prático/indicadoressegurancapublicauf.xlsx",sheet_name='Ocorrências')
data_ocorrencias

,UF,Tipo Crime,Ano,Mês,Ocorrências
0,Acre,Estupro,2022.0,janeiro,31.0
1,Acre,Furto de veículo,2022.0,janeiro,50.0
2,Acre,Homicídio doloso,2022.0,janeiro,9.0
3,Acre,Lesão corporal seguida de morte,2022.0,janeiro,0.0
4,Acre,Roubo a instituição financeira,2022.0,janeiro,0.0
...,...,...,...,...,...
20441,Tocantins,Roubo a instituição financeira,2015.0,dezembro,6.0
20442,Tocantins,Roubo de carga,2015.0,dezembro,1.0
20443,Tocantins,Roubo de veículo,2015.0,dezembro,55.0
20444,Tocantins,Roubo seguido de morte (latrocínio),2015.0,dezembro,2.0


In [5]:
data_vitimas = pd.read_excel("/content/drive/MyDrive/Análise de dados/UND 2/Trabalho prático/indicadoressegurancapublicauf.xlsx",sheet_name='Vítimas')
data_vitimas

,UF,Tipo Crime,Ano,Mês,Sexo da Vítima,Vítimas
0,Acre,Homicídio doloso,2022.0,janeiro,Feminino,1.0
1,Acre,Homicídio doloso,2022.0,janeiro,Masculino,8.0
2,Acre,Homicídio doloso,2022.0,janeiro,Sexo NI,0.0
3,Acre,Lesão corporal seguida de morte,2022.0,janeiro,Feminino,0.0
4,Acre,Lesão corporal seguida de morte,2022.0,janeiro,Masculino,0.0
...,...,...,...,...,...,...
17430,Tocantins,Roubo seguido de morte (latrocínio),2015.0,junho,Masculino,1.0
17431,Tocantins,Roubo seguido de morte (latrocínio),2015.0,julho,Masculino,1.0
17432,Tocantins,Roubo seguido de morte (latrocínio),2015.0,agosto,Masculino,1.0
17433,Tocantins,Roubo seguido de morte (latrocínio),2015.0,outubro,Masculino,2.0


# Vizualização dos daods
A vizualisação dos dados antes e depois da cluterização poderá ser feita por meio do dashboard feito utilizando o Power Bi .O quadro estará disponível junto com esse notebook no mesmo repositório no GitHub. 

# Aplicando o algoritmo de clusterização 


In [16]:
from sklearn import metrics

for k in range(2, 10):

  kmeans = KMeans(n_clusters=k)
  kmeans.fit(data_ocorrencias[['Ocorrências']])
  score_mean = metrics.silhouette_score(data_ocorrencias[['Ocorrências']], kmeans.labels_, metric='euclidean')
  print(k, score_mean)

2 0.9555921166074182
3 0.8577003499382452
4 0.8484278359336169
5 0.7521311686073353
6 0.7533746117466757
7 0.7502651613151318
8 0.7466548397792157
9 0.7075769236073941


In [17]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(data_ocorrencias[["Ocorrências"]])

KMeans(n_clusters=10)

In [20]:
clusters = kmeans.labels_.tolist()
clusters_dict = { 'UF': list(data_ocorrencias.UF), 'cluster': clusters}
dataframe_clusters = pd.DataFrame(clusters_dict, columns = ['cluster','UF'])
dataframe_clusters

,cluster,UF
0,6,Acre
1,6,Acre
2,6,Acre
3,6,Acre
4,6,Acre
...,...,...
20441,6,Tocantins
20442,6,Tocantins
20443,6,Tocantins
20444,6,Tocantins
